In [1]:
import requests
from urllib.parse import urlparse
import numpy as np
import pandas as pd
import geopandas
import re

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
data = pd.read_csv('subway_gate_data.csv', encoding='utf-8')
print(data.shape)
data.head()

(2499, 3)


,지하철역ID,지하철역명,출입구번호
0,1067080129,천마산,1
1,1067080129,천마산,2
2,1067080130,마석,1
3,1067080130,마석,2
4,1067080131,대성리,1


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2499 entries, 0 to 2498
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   지하철역ID  2499 non-null   int64 
 1   지하철역명   2499 non-null   object
 2   출입구번호   2499 non-null   object
dtypes: int64(1), object(2)
memory usage: 58.7+ KB


In [4]:
# 주소 검색용 컬럼 생성
data['검색키워드'] = data['지하철역명'] + '역 ' + data['출입구번호'] + '번 출구'
data.head()

,지하철역ID,지하철역명,출입구번호,검색키워드
0,1067080129,천마산,1,천마산역 1번 출구
1,1067080129,천마산,2,천마산역 2번 출구
2,1067080130,마석,1,마석역 1번 출구
3,1067080130,마석,2,마석역 2번 출구
4,1067080131,대성리,1,대성리역 1번 출구


In [5]:
len(data['검색키워드'])

2499

In [6]:
search_list = data['검색키워드'].tolist()
search_list

['천마산역 1번 출구',
 '천마산역 2번 출구',
 '마석역 1번 출구',
 '마석역 2번 출구',
 '대성리역 1번 출구',
 '대성리역 2번 출구',
 '청평역 1번 출구',
 '청평역 2번 출구',
 '상천역 1번 출구',
 '가평역 1번 출구',
 '굴봉산역 1번 출구',
 '굴봉산역 2번 출구',
 '백양리역 1번 출구',
 '강촌역 1번 출구',
 '김유정역 1번 출구',
 '남춘천역 1번 출구',
 '남춘천역 2번 출구',
 '남춘천역 3번 출구',
 '춘천역 1번 출구',
 '춘천역 2번 출구',
 '계양역 1번 출구',
 '귤현역 1번 출구',
 '박촌역 1번 출구',
 '박촌역 2번 출구',
 '박촌역 3번 출구',
 '박촌역 4번 출구',
 '임학역 1번 출구',
 '임학역 2번 출구',
 '임학역 3번 출구',
 '임학역 4번 출구',
 '계산역 1번 출구',
 '계산역 2번 출구',
 '계산역 3번 출구',
 '계산역 4번 출구',
 '계산역 5번 출구',
 '계산역 6번 출구',
 '원흥역 1번 출구',
 '원흥역 2번 출구',
 '원흥역 3번 출구',
 '원흥역 4번 출구',
 '원흥역 5번 출구',
 '원흥역 6번 출구',
 '원흥역 7번 출구',
 '원흥역 8번 출구',
 '강매역 1번 출구',
 '서울역 3번 출구',
 '서울역 4번 출구',
 '공덕역 10번 출구',
 '공덕역 9번 출구',
 '홍대입구역 3번 출구',
 '홍대입구역 4번 출구',
 '홍대입구역 5번 출구',
 '홍대입구역 6번 출구',
 '홍대입구역 7번 출구',
 '디지털미디어시티역 7번 출구',
 '디지털미디어시티역 8번 출구',
 '디지털미디어시티역 9번 출구',
 '김포공항역 4번 출구',
 '검암역 1번 출구',
 '청라국제도시역 1번 출구',
 '청라국제도시역 2번 출구',
 '공항화물청사역 1번 출구',
 '공항화물청사역 2번 출구',
 '인천공항1터미널역 1번 출구',
 '인천공항1터미널역 2번 출구',
 '인천공항1터미널역 3번 출구',


In [7]:
import requests

# Kakao API 키
api_key = os.environ.get("KAKAO_API_KEY")

# 위도, 경도 검색 함수 정의
def lat_lon(address):
    url = f"https://dapi.kakao.com/v2/local/search/keyword.json?&query={address}"
    headers = {"Authorization": f"KakaoAK {api_key}"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        result = response.json()
        if result.get("documents"):
            first_result = result["documents"][0]
            place_name = first_result["place_name"]
            latitude = first_result["y"]
            longitude = first_result["x"]
            return (latitude, longitude)
        else:
            print(f"검색 결과가 없습니다: {address}")
    else:
        print(f"API 요청 실패: {response.status_code}")

In [8]:
# 위도, 경도 검색 함수 반복 적용
location_list = []
for i, search_term in enumerate(search_list):
    print(f"Processing {i+1}/{len(search_list)}: {search_term}")
    lat_lon_result = lat_lon(search_term)
    location_list.append(lat_lon_result)

Processing 1/2499: 천마산역 1번 출구
Processing 2/2499: 천마산역 2번 출구
Processing 3/2499: 마석역 1번 출구
Processing 4/2499: 마석역 2번 출구
Processing 5/2499: 대성리역 1번 출구
Processing 6/2499: 대성리역 2번 출구
Processing 7/2499: 청평역 1번 출구
Processing 8/2499: 청평역 2번 출구
Processing 9/2499: 상천역 1번 출구
Processing 10/2499: 가평역 1번 출구
Processing 11/2499: 굴봉산역 1번 출구
Processing 12/2499: 굴봉산역 2번 출구
Processing 13/2499: 백양리역 1번 출구
Processing 14/2499: 강촌역 1번 출구
Processing 15/2499: 김유정역 1번 출구
Processing 16/2499: 남춘천역 1번 출구
Processing 17/2499: 남춘천역 2번 출구
Processing 18/2499: 남춘천역 3번 출구
Processing 19/2499: 춘천역 1번 출구
Processing 20/2499: 춘천역 2번 출구
Processing 21/2499: 계양역 1번 출구
Processing 22/2499: 귤현역 1번 출구
Processing 23/2499: 박촌역 1번 출구
Processing 24/2499: 박촌역 2번 출구
Processing 25/2499: 박촌역 3번 출구
Processing 26/2499: 박촌역 4번 출구
Processing 27/2499: 임학역 1번 출구
Processing 28/2499: 임학역 2번 출구
Processing 29/2499: 임학역 3번 출구
Processing 30/2499: 임학역 4번 출구
Processing 31/2499: 계산역 1번 출구
Processing 32/2499: 계산역 2번 출구
Processing 33/2499: 계산역 3번 출구
Processi

In [9]:
location = pd.DataFrame(location_list, columns=['위도','경도'])
print(location.shape)
location.head()

(2499, 2)


,위도,경도
0,37.65930925674902,127.28560473195162
1,37.65910145706316,127.28584192923722
2,37.65222456883558,127.3116977806786
3,37.6531162889649,127.31179216474
4,37.68392192187381,127.37904610748313


In [10]:
station_gate = pd.concat([data, location], axis=1)
print(station_gate.shape)
station_gate.head()

(2499, 6)


,지하철역ID,지하철역명,출입구번호,검색키워드,위도,경도
0,1067080129,천마산,1,천마산역 1번 출구,37.65930925674902,127.28560473195162
1,1067080129,천마산,2,천마산역 2번 출구,37.65910145706316,127.28584192923722
2,1067080130,마석,1,마석역 1번 출구,37.65222456883558,127.3116977806786
3,1067080130,마석,2,마석역 2번 출구,37.6531162889649,127.31179216474
4,1067080131,대성리,1,대성리역 1번 출구,37.68392192187381,127.37904610748313


In [11]:
station_gate.to_csv('subway_gate_location_data.csv', index=False)

In [19]:
import json

# 서울 경계 좌표
seoul_boundary = {
    'min_lon': 126.734086,
    'max_lon': 127.269311,
    'min_lat': 37.413294,
    'max_lat': 37.715133
}

# 서울 내 출입구 데이터와 외부 출입구 데이터를 담을 리스트
success_to_process_gate = []
fail_to_process_gate = []

# subway_gate_location_data.csv 파일 불러오기
station_gate = pd.read_csv('subway_gate_location_data.csv')

# 데이터 처리 및 분류
for index, row in station_gate.iterrows():
    lon, lat = row['경도'], row['위도']

    if seoul_boundary['min_lon'] <= lon <= seoul_boundary['max_lon'] and \
       seoul_boundary['min_lat'] <= lat <= seoul_boundary['max_lat']:
        success_to_process_gate.append({
            '지하철역명': row['지하철역명'],
            '출입구번호': row['출입구번호'],
            '위도': row['위도'],
            '경도': row['경도']
        })
    else:
        fail_to_process_gate.append({
            '지하철역명': row['지하철역명'],
            '출입구번호': row['출입구번호'],
            '위도': row['위도'],
            '경도': row['경도']
        })

# JSON 파일로 저장
with open('gate_location_in_seoul.json', 'w', encoding='utf-8') as success_file:
    json.dump(success_to_process_gate, success_file, ensure_ascii=False, indent=4)

with open('gate_location_not_in_seoul.json', 'w', encoding='utf-8') as fail_file:
    json.dump(fail_to_process_gate, fail_file, ensure_ascii=False, indent=4)

print("데이터 처리 및 JSON 파일 저장이 완료되었습니다.")

데이터 처리 및 JSON 파일 저장이 완료되었습니다.


In [20]:
total_data_count = len(station_gate)
print(f'station_gate_location_data.csv의 총 데이터 개수: {total_data_count}')
print(f'gate_location_in_seoul.json 파일의 총 데이터 개수: {len(success_to_process_gate)}')
print(f'gate_location_not_in_seoul.json 파일의 총 데이터 개수: {len(fail_to_process_gate)}')

station_gate_location_data.csv의 총 데이터 개수: 2499
gate_location_in_seoul.json 파일의 총 데이터 개수: 1961
gate_location_not_in_seoul.json 파일의 총 데이터 개수: 538


In [21]:
import json

# gate_location_not_in_seoul.json 파일 로드
with open('gate_location_not_in_seoul.json', 'r', encoding='utf-8') as not_in_seoul_file:
    not_in_seoul_data = json.load(not_in_seoul_file)

# 중복을 제거한 지하철역명 추출
not_in_seoul_station_names = list(set(entry['지하철역명'] for entry in not_in_seoul_data))

# not_in_seoul_station_name.json 파일로 저장
with open('not_in_seoul_station_name.json', 'w', encoding='utf-8') as not_in_seoul_station_name_file:
    json.dump(not_in_seoul_station_names, not_in_seoul_station_name_file, ensure_ascii=False, indent=4)

print('not_in_seoul_station_name.json 파일이 생성되었습니다.')

not_in_seoul_station_name.json 파일이 생성되었습니다.
